In [1]:
# CELL 1 - Install Dependencies

# Install topic modeling, NLP, and visualization libraries
!pip install -q gensim pyldavis nltk spacy

# Download spaCy's small English language model
# This provides tokenization, lemmatization, stopwords, etc.
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# CELL 2 — Imports & Paths

# ---------------- BASIC UTILITIES ----------------
import os              # File and directory handling
import re              # Regular expressions for text cleaning
import pickle          # Save and load Python objects
import json            # Save topic labels in readable format
import warnings        # Control warning messages

# ---------------- DATA HANDLING ----------------
import pandas as pd    # DataFrames and CSV handling

# ---------------- NLP ----------------
import nltk            # Tokenizers (used indirectly)
import spacy           # Lemmatization, stopwords

# ---------------- TOPIC MODELING ----------------
from gensim import corpora, models
from gensim.models import CoherenceModel


# Path to dataset attached on Kaggle (READ ONLY)
DATASET_PATH = "/kaggle/input/ansar1/Ansar1.txt"

# Path where all outputs will be saved (WRITE ENABLED)
SAVE_PATH = "/kaggle/working/LDA_Models"

# Create directory if it does not exist
# This ensures outputs survive after notebook execution
os.makedirs(SAVE_PATH, exist_ok=True)

print("Output directory ready:", SAVE_PATH)


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


Output directory ready: /kaggle/working/LDA_Models


In [3]:
# # to reuse model later

# # in new kaggle notebook
# import pickle

# with open("/kaggle/input/your-notebook-output/lda_model.pkl", "rb") as f:
#     lda_model = pickle.load(f)

In [4]:
# CELL 3 — Load Dataset

# Load the dataset (tab-separated file)
df = pd.read_csv(
    DATASET_PATH,
    sep="\t",             # Dataset is TSV
    on_bad_lines="skip"   # Skip corrupted lines safely
)

# Extract message text and remove null values
documents = df["Message"].dropna().tolist()

print(f"Total documents loaded: {len(documents)}")


Total documents loaded: 28866


In [5]:
# CELL 4 — NLP Setup

# Download NLTK tokenizer resources (quiet mode)
nltk.download("punkt", quiet=True)

# Load spaCy English model
# Disable parser and NER to reduce computation time
nlp = spacy.load(
    "en_core_web_sm",
    disable=["parser", "ner"]
)

print("spaCy NLP pipeline loaded successfully.")


spaCy NLP pipeline loaded successfully.


In [6]:
# CELL 5 — Preprocessing Function (FINAL, CORRECT)

def preprocess_spacy(text):
    """
    Cleans and preprocesses a single document:
    1. Removes URLs
    2. Keeps only English letters
    3. Lemmatizes words
    4. Removes stopwords
    5. Removes short and non-alphabetic tokens
    """

    # Convert to string (safety)
    text = str(text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text)

    # Remove numbers, symbols, and non-English characters
    text = re.sub(r"[^a-zA-Z\s]", " ", text)

    # Process text using spaCy
    doc = nlp(text)

    # Return clean tokens
    return [
        token.lemma_.lower()      # Lemmatized lowercase word
        for token in doc
        if token.is_alpha         # Only alphabetic tokens
        and not token.is_stop     # Remove stopwords
        and len(token.lemma_) > 2 # Remove very short words
    ]


# Apply preprocessing to all documents
processed_docs = [preprocess_spacy(doc) for doc in documents]

print("Text preprocessing completed.")


Text preprocessing completed.


In [7]:
# CELL 6 — Dictionary & Corpus

# Create dictionary (word → id mapping)
dictionary = corpora.Dictionary(processed_docs)

# Remove:
# - Words appearing in <5 documents (noise)
# - Words appearing in >50% documents (too common)
dictionary.filter_extremes(
    no_below=5,
    no_above=0.5
)

# Convert documents to Bag-of-Words representation
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

print("Dictionary size:", len(dictionary))
print("Corpus creation successful.")


Dictionary size: 16062
Corpus creation successful.


In [8]:
# CELL 7 — Train LDA Model

# Number of topics to extract
NUM_TOPICS = 10

# Train LDA model using Gensim
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=NUM_TOPICS,
    random_state=100,   # Reproducibility
    passes=10           # More passes = better convergence
)

print("LDA model training completed.")


LDA model training completed.


In [9]:
# NEW CELL 7.1 — PRINT THE 10 TOPICS
print("\n--- Top 10 Topics ---")

for topic_id in range(lda_model.num_topics):
    words = lda_model.show_topic(topic_id, topn=10)
    clean_words = [word for word, _ in words]
    print(f"Topic {topic_id}: {clean_words}")


--- Top 10 Topics ---
Topic 0: ['somalia', 'somali', 'government', 'mogadishu', 'say', 'shabaab', 'town', 'group', 'islamist', 'shabab']
Topic 1: ['islamic', 'people', 'muslims', 'jihad', 'muslim', 'group', 'state', 'islam', 'god', 'country']
Topic 2: ['pakistan', 'taliban', 'say', 'attack', 'militant', 'pakistani', 'official', 'government', 'kill', 'army']
Topic 3: ['israel', 'israeli', 'hamas', 'gaza', 'pirate', 'palestinian', 'say', 'ship', 'palestinians', 'fire']
Topic 4: ['kill', 'say', 'police', 'attack', 'soldier', 'wound', 'bomb', 'iraq', 'iraqi', 'force']
Topic 5: ['say', 'man', 'year', 'tell', 'woman', 'know', 'court', 'people', 'case', 'family']
Topic 6: ['mujahideen', 'kill', 'emirate', 'afghanistan', 'terrorist', 'report', 'islamic', 'province', 'destroy', 'district']
Topic 7: ['weapon', 'world', 'war', 'nuclear', 'les', 'cia', 'russia', 'company', 'people', 'work']
Topic 8: ['allah', 'brother', 'quote', 'post', 'mujahideen', 'know', 'originally', 'video', 'like', 'good']

In [10]:
# CELL 8 — Compute & Save Coherence Score

# Initialize coherence model using C_v metric
coherence_model = CoherenceModel(
    model=lda_model,
    texts=processed_docs,
    dictionary=dictionary,
    coherence="c_v"
)

# Compute coherence score
coherence_score = coherence_model.get_coherence()

print(f"Coherence Score (C_v): {coherence_score:.4f}")


# Save coherence score for reporting and reproducibility
with open(f"{SAVE_PATH}/coherence_score.txt", "w") as f:
    f.write(f"Coherence Score (C_v): {coherence_score:.4f}")


Coherence Score (C_v): 0.5271


In [11]:
# NEW CELL 8.1 — TOPIC LABEL DEFINITION

topic_labels = {
    0: "Somalia & Shabaab News",
    1: "General Islamic Topics",
    2: "Pakistan & Taliban Conflict",
    3: "Israel-Palestine Conflict",
    4: "Iraq Conflict & Attacks",
    5: "Personal/Court Cases",
    6: "Afghanistan Mujahideen Reports",
    7: "Global/Nuclear Weapons",
    8: "Forum Chat & Religious Posts",
    9: "US/Afghanistan Policy (Obama)"
}

print("Custom topic labels assigned.")


Custom topic labels assigned.


In [12]:
# NEW CELL 8.2 — DOCUMENT CATEGORIZATION (IMPORTANT)

categorized_docs = []

for i in range(len(corpus)):
    # Get topic probability distribution for document
    topic_distribution = lda_model[corpus[i]]

    if not topic_distribution:
        continue

    # Select topic with highest probability
    dominant_topic = max(topic_distribution, key=lambda x: x[1])

    categorized_docs.append({
        "Original Text": documents[i],
        "Category ID": dominant_topic[0],
        "Category Label": topic_labels.get(dominant_topic[0], "Unknown"),
        "Probability": round(dominant_topic[1], 4)
    })

# Convert results to DataFrame
df_results = pd.DataFrame(categorized_docs)

print("Document categorization completed.")
df_results.head()


Document categorization completed.


,Original Text,Category ID,Category Label,Probability
0,In the Name of Allah the Most Merciful الإخوة ...,8,Forum Chat & Religious Posts,0.6436
1,http://www.timesonline.co.uk/tol/new...cle6139...,7,Global/Nuclear Weapons,0.4031
2,http://www.honoluluadvertiser.com/ar...9042203...,4,Iraq Conflict & Attacks,0.7843
3,Iraq Office of the Assistant Secretary of Defe...,4,Iraq Conflict & Attacks,0.8762
4,As'Salaamu alaikum. Because the war is over an...,4,Iraq Conflict & Attacks,0.5708


In [13]:
# NEW CELL 8.3 — SAVE CATEGORIZATION RESULTS

df_results.to_csv(
    f"{SAVE_PATH}/categorized_documents.csv",
    index=False
)

print("Categorized documents saved to disk.")


Categorized documents saved to disk.


In [14]:
# CELL 9 — Save Trained Model & Supporting Files

# Save LDA model
with open(f"{SAVE_PATH}/lda_model.pkl", "wb") as f:
    pickle.dump(lda_model, f)

# Save dictionary
with open(f"{SAVE_PATH}/dictionary.pkl", "wb") as f:
    pickle.dump(dictionary, f)

# Save corpus
with open(f"{SAVE_PATH}/corpus.pkl", "wb") as f:
    pickle.dump(corpus, f)

# Save original documents
with open(f"{SAVE_PATH}/documents.pkl", "wb") as f:
    pickle.dump(documents, f)

# Save topic labels
with open(f"{SAVE_PATH}/topic_labels.json", "w") as f:
    json.dump(topic_labels, f, indent=4)

print("Model and metadata saved successfully.")



Model and metadata saved successfully.


In [15]:
# CELL 10 — pyLDAvis

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Suppress non-critical warnings for clean output
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Enable notebook visualization
pyLDAvis.enable_notebook()

# Prepare visualization
vis_data = gensimvis.prepare(
    lda_model,
    corpus,
    dictionary
)

vis_data


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.046165  0.106819       1        1  19.845048
5      0.127834  0.073895       2        1  13.754032
4     -0.192664 -0.012073       3        1  13.533271
8      0.220648 -0.187287       4        1  13.329601
2     -0.113802  0.064827       5        1  10.717838
1      0.174389 -0.030341       6        1   8.616110
6     -0.185911 -0.297042       7        1   7.275008
7      0.190024  0.076652       8        1   5.806438
0     -0.087309  0.045526       9        1   4.656158
3     -0.179375  0.159024      10        1   2.466496, topic_info=            Term          Freq         Total Category  logprob  loglift
2          allah  18637.000000  18637.000000  Default  30.0000  30.0000
11    mujahideen  10219.000000  10219.000000  Default  29.0000  29.0000
341      taliban  11041.000000  11041.000000  Default  28.0000  28.0000
313     pakistan   8711.000000   8711.000000  Default  27.0000  27.0000
66           say  36588.000000  36588.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
388       report    357.688356   9286.955878  Topic10  -5.0603   0.4457
817   helicopter    239.921802    848.537566  Topic10  -5.4597   2.4392
115       border    255.358056   2626.803951  Topic10  -5.3973   1.3715
626         hold    243.137059   2609.164605  Topic10  -5.4463   1.3292
1033        west    242.318666   2183.730762  Topic10  -5.4497   1.5038

[723 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
5526       7  0.996013       abad
5026       9  0.997211  abdullahi
1          2  0.107182        abu
1          3  0.014616        abu
1          4  0.501264        abu
...      ...       ...        ...
3062       8  0.010865   zawahiri
3063       4  0.163641    zionist
3063       6  0.835050    zionist
10895      6  0.986458   zionists
3017       2  0.992969   zubaydah

[1800 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 5, 9, 3, 2, 7, 8, 1, 4])